In [1]:
import os
import fitz
import re
import json

from ollama import Client
import faiss
import pandas as pd
import numpy as np
import Stemmer
from tqdm import tqdm
import gradio as gr

from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.retrievers import BaseRetriever, QueryFusionRetriever
from llama_index.core.schema import TextNode, NodeWithScore
from llama_index.retrievers.bm25 import BM25Retriever

# Connect to Ollama Server

In [2]:
client = Client(
  host='http://localhost:11434',
)

# Ingestion

In [3]:
# Path to the dataset folder
DATASET_PATH = 'extracted_text.json'

def get_text_and_metadata(input_path):
    """Load text and metadata from a file and perform chunking."""
    with open(input_path, "r", encoding="utf-8") as f:
        extracted_data = json.load(f)

    texts = []
    metadata = []

    for entry in extracted_data:
        text = entry["text"]
        source_metadata = {
            "source": entry["source"],
            "folder": entry["folder"],
            "file_name": entry["file_name"],
            "page": entry["page"],
            "title": entry["title"],
            "url": entry["url"]
        }
        texts.append(text)
        metadata.append(source_metadata)

    return texts, metadata

In [4]:
docs, metadatas = get_text_and_metadata(DATASET_PATH)

In [5]:
documents = [Document(text=docs[t], metadata=metadatas[t]) for t in range(len(docs))]
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

# Embedding and Retrieval

## Dense via FAISS

In [6]:
def generate_embeddings(nodes, client, model):
    # Generate embeddings for documents using Ollama
    for doc in tqdm(nodes):
        response = client.embeddings(prompt=doc.text, model=model)
        doc.embedding = response["embedding"]
    return nodes

In [7]:
class FaissIndexer:
    """
    Faiss-based indexer for efficient similarity search using inner-product (cosine) similarity.

    This class handles the creation and management of a FAISS index from node embeddings.
    
    :ivar faiss_index: The FAISS index for storing and querying embeddings.
    :vartype faiss_index: faiss.IndexFlatIP
    :ivar embedding_dim: Dimensionality of the embeddings.
    :vartype embedding_dim: int
    """

    def __init__(self):
        """
        Initialize the FaissIndexer class.

        :ivar faiss_index: The FAISS index, initialized as None.
        :ivar embedding_dim: The dimension of embeddings, initialized as None.
        """
        self.faiss_index = None
        self.embedding_dim = None

    def normalize_embeddings(self, embeddings):
        """
        Normalize embeddings to have unit L2 norm.

        :param embeddings: Array of embeddings to normalize.
        :type embeddings: np.ndarray
        :return: Normalized embeddings.
        :rtype: np.ndarray
        """
        return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    def build_index(self, nodes):
        """
        Build the FAISS index from a list of nodes containing embeddings.

        :param nodes: List of nodes, where each node contains an `embedding` attribute.
        :type nodes: list
        :raises ValueError: If the nodes list is empty or embeddings are inconsistent.
        """
        if not nodes:
            raise ValueError("Nodes list cannot be empty.")
        
        embeddings = np.array([np.array(node.embedding) for node in nodes])
        normalized_embeddings = self.normalize_embeddings(embeddings)

        self.embedding_dim = normalized_embeddings[0].shape[0]
        self.faiss_index = faiss.IndexFlatIP(self.embedding_dim)  # Inner-product similarity
        self.faiss_index.add(normalized_embeddings)

    def get_index(self):
        """
        Get the FAISS index instance.

        :return: The FAISS index used for similarity search.
        :rtype: faiss.IndexFlatIP
        :raises ValueError: If the index has not been built.
        """
        if self.faiss_index is None:
            raise ValueError("Index has not been built yet. Call 'build_index' first.")
        return self.faiss_index

In [8]:
class FAISSVectorStoreRetriever(BaseRetriever):
    def __init__(self, faiss_index, documents):
        """
        Initialize the FAISS retriever.
        :param faiss_index: The FAISS index containing precomputed embeddings.
        :param documents: List of document chunks.
        :param embeddings: Precomputed embeddings corresponding to the document chunks.
        """
        self.faiss_index = faiss_index
        self.documents = documents

    def _retrieve(self, query_embedding, top_k=5):
        """
        Retrieve the top-k nearest neighbors using the FAISS index.
        :param query_embedding: The embedding of the query.
        :param top_k: Number of top results to retrieve.
        """

        norm_query_embedding = np.array([query_embedding])
        norm_query_embedding /= np.linalg.norm(norm_query_embedding, axis=1, keepdims=True)

        distances, indices = self.faiss_index.search(norm_query_embedding, top_k)
        retrieved_docs = [
            NodeWithScore(node=self.documents[idx], score=1 - dist)
            for idx, dist in zip(indices[0], distances[0])
            if idx != -1
        ]
        return retrieved_docs

In [9]:
# embedding
nodes_embed = generate_embeddings(nodes, client, "mxbai-embed-large")

100%|██████████| 12302/12302 [41:35<00:00,  4.93it/s] 


In [10]:
#indexing
index = FaissIndexer()
index.build_index(nodes_embed)
faiss_index = index.get_index()

faiss_retriever = FAISSVectorStoreRetriever(faiss_index=faiss_index,documents=nodes_embed) 

## Sparse Embedding via BM25

In [11]:
# bm25_retriever = BM25Retriever.from_defaults(
#     nodes=nodes,
#     similarity_top_k=5,
#     stemmer=Stemmer.Stemmer("english"),
#     language="english",
# )

## Hybrid Retrieval via Reciprocal Rank

In [12]:
def hybrid_embedding(results: dict, top_k: int):
    x = QueryFusionRetriever
    ranked_results = QueryFusionRetriever._reciprocal_rerank_fusion(x, results)
    return ranked_results[:top_k]

# Post Retrieval

## Summarization

In [13]:
def summarize_each_chunk(nodes, client, query, model="llama3.3", parent=False):
    if parent:
        chunks = [doc.text for doc in nodes]
    else:
        chunks = [doc.node.text for doc in nodes]
    summaries = []
    
    for i, chunk in enumerate(chunks):
        prompt = f"""
        Summarize the following text in one concise paragraph, focusing on key points relevant to the query: "{query}".
        
        - Emphasize information directly related to the query.
        - Exclude unrelated, redundant, or speculative details.
        - Do NOT introduce new information or answer the query itself. 
        
        Text:
        {chunk}
        
        Summary:
        """
        
        response = client.generate(model=model, prompt=prompt)
        summary = response['response'].strip()
        summaries.append(summary)

    return summaries

# Generation

In [14]:


def generate_response_with_notice(summaries, query, choices, client, model="llama3.3"):
    # Combine summaries into context block
    context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Use the following summarized information to answer the query accurately and concisely. 
    DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
    If the information is not sufficient to fully address the query, respond ONLY with:
    "The available information is insufficient to provide a complete answer to this query."

    Summarized Context:
    {context}
    
    Query:
    {query}
    {choices}
    
    \nPlease answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED.
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

# Querying

## Query Transforms

In [27]:
import re
import json

prompt = f"""
You are an expert assistant. Below are examples of how to decompose a query
into six sub-queries, and also provide short answers for each sub-query.
Finally, place these into a JSON array.

Example #1:
USER: "Why is the sky blue?"
SUB-QUERY 1: "Explain Rayleigh scattering"
SUB-QUERY 2: "How light interacts with molecules?"
SUB-QUERY 3: "How human eyes perceive color?"
SUB-QUERY 4: "What factors affect the intensity of Rayleigh scattering?"
SUB-QUERY 5: "Why does the sky change color during sunset?"
SUB-QUERY 6: "What role does atmospheric composition play in sky color?"


FINAL ANSWER (JSON array):
{{
    "main_query": "Why is the sky blue?",
    "sub_queries": [
        {{ "sub_query": "Explain Rayleigh scattering" }},
        {{ "sub_query": "How light interacts with molecules?" }},
        {{ "sub_query": "How human eyes perceive color?" }},
        {{ "sub_query": "What factors affect the intensity of Rayleigh scattering?" }},
        {{ "sub_query": "Why does the sky change color during sunset?" }},
        {{ "sub_query": "What role does atmospheric composition play in sky color?" }}
    ]
}}

Now your turn:
USER: "{"What are the requirements in getting a license?"}"

FINAL ANSWER (JSON array):
"""

response = client.generate(model="llama3.3", prompt=prompt)
print(response.response)

# Updated regex to extract the entire JSON object including main_query
match = re.search(r'\{\s*("main_query"\s*:\s*".*?"\s*,\s*"sub_queries"\s*:\s*\[.*?\])\s*\}', response.response, re.S)
if match:
    json_text = match.group(0)
    try:
        parsed_json = json.loads(json_text)
        print(json.dumps(parsed_json, indent=4))
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        print(json_text)
else:
    print("No JSON found.")

To address the user's query, "What are the requirements in getting a license?", we need to break it down into more specific sub-queries that cover various aspects of obtaining a license. Here's how we can decompose this query:

1. **Understanding License Types**: The first step is to identify what type of license the user is referring to, as different licenses (e.g., driver's license, professional license, business license) have different requirements.

2. **Educational Requirements**: Many licenses require applicants to complete specific educational courses or have certain degrees.

3. **Examination and Assessment**: Most licensing processes involve passing an examination or assessment that tests the applicant's knowledge or skills in the relevant area.

4. **Background Checks**: For some licenses, particularly those related to security, childcare, or healthcare, background checks may be necessary.

5. **Application and Fees**: There's usually a formal application process involving pa

## Query Generation

In [28]:
docstore = {}

# Store documents using full metadata as the key
for doc in documents:
    key = tuple(doc.metadata.items())  # Convert metadata to tuple for hashable key
    docstore[key] = doc

In [29]:
def get_document_by_chunk_metadata(chunk_node):
    # Convert chunk metadata to tuple for matching
    metadata_key = tuple(chunk_node.metadata.items())

    # Retrieve document from docstore
    document = docstore.get(metadata_key)
    return document

In [30]:
def remove_duplicate_documents(doc_list):
    seen_ids = set()
    unique_docs = []

    for doc in doc_list:
        if doc.doc_id not in seen_ids:
            seen_ids.add(doc.doc_id)
            unique_docs.append(doc)

    return unique_docs

In [31]:
import json

def gen_query(query, choices, top_k, client, mode='hybrid', summary=False, model="llama3.3", chunks_only=False):
    prompt = f"""
    You are an expert assistant. Below are examples of how to decompose a query
    into six sub-queries, and also provide short answers for each sub-query.
    Finally, place these into a JSON array.

    Example #1:
    USER: "Why is the sky blue?"
    SUB-QUERY 1: "Explain Rayleigh scattering"
    SUB-QUERY 2: "How light interacts with molecules?"
    SUB-QUERY 3: "How human eyes perceive color?"
    SUB-QUERY 4: "What factors affect the intensity of Rayleigh scattering?"
    SUB-QUERY 5: "Why does the sky change color during sunset?"
    SUB-QUERY 6: "What role does atmospheric composition play in sky color?"


    FINAL ANSWER (JSON array):
    {{
        "main_query": "Why is the sky blue?",
        "sub_queries": [
            {{ "sub_query": "Explain Rayleigh scattering" }},
            {{ "sub_query": "How light interacts with molecules?" }},
            {{ "sub_query": "How human eyes perceive color?" }},
            {{ "sub_query": "What factors affect the intensity of Rayleigh scattering?" }},
            {{ "sub_query": "Why does the sky change color during sunset?" }},
            {{ "sub_query": "What role does atmospheric composition play in sky color?" }}
        ]
    }}

    Now your turn:
    USER: "{query}"

    FINAL ANSWER (JSON array):
    """


    response_text = client.generate(model=model, prompt=prompt)

    match = re.search(r'\{\s*("main_query"\s*:\s*".*?"\s*,\s*"sub_queries"\s*:\s*\[.*?\])\s*\}', response_text.response, re.S)
    if match:
        json_text = match.group(0)
        try:
            parsed_json = json.loads(json_text)
            print(json.dumps(parsed_json, indent=4))
            embed_result = client.embeddings(prompt=json_text, model="mxbai-embed-large")
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
            print(json_text)
            embed_result = client.embeddings(prompt=query, model="mxbai-embed-large")
    else:
        print("No JSON found.")
        embed_result = client.embeddings(prompt=query, model="mxbai-embed-large")

    # Return both the parsed sub-queries/answers and the embedding vector
    query_embedding = embed_result["embedding"]

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.3', query=query,parent=parent_flag)
        context = summaries

    answer = generate_response_with_notice(context, query, choices, client, model=model)

    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references)

# Evaluation

In [32]:
# Generate prompts dynamically
def generate_choices(row):
    options = []
    for choice in ['A', 'B', 'C', 'D', 'E']:
        # Check for NaN or blank values
        if pd.notna(row[choice]) and row[choice] != '':
            options.append(f"{choice}. {row[choice]}")
    
    # Construct the prompt with few-shot examples
    choices = "\n".join(options)
    
    return choices

In [33]:
from sklearn.model_selection import train_test_split
# Load the Excel file
file_path = '/mnt/c/Users/Jeryl Salas/Documents/AI 351/Project/LTO_EXAM.csv'
df = pd.read_csv(file_path)
df['Choices'] = df.apply(generate_choices, axis=1)
# Split the data into test (80%) and holdout validation (20%)
test_df, holdout_df = train_test_split(df, test_size=0.33, random_state=42)

# Display the first few rows of each set
print("Testing Data:")
display(test_df.head())

print("\nHoldout Validation Data:")
display(holdout_df.head())

test_df.to_csv('/mnt/c/Users/Jeryl Salas/Documents/AI 351/Project/test_data.csv', index=False)
holdout_df.to_csv('/mnt/c/Users/Jeryl Salas/Documents/AI 351/Project/holdout_data.csv', index=False)

Testing Data:


,Question,A,B,C,D,E,Answer,Choices
69,"If the driver is turning left, he must: (U-tur...",have the right of way,do so slowly with caution,yield to approaching cars,NaN,NaN,C,A. have the right of way\nB. do so slowly with...
5,"When a vehicle starts to skid, what should the...",Immediately step on the brakes,Hold firmly on to the wheel while slowing down...,Turn the wheels tp the opposite the direction ...,NaN,NaN,B,A. Immediately step on the brakes\nB. Hold fir...
42,"If you are parking uphill without a curb, turn...",edge of the street,other side of the street,middle of the street,NaN,NaN,A,A. edge of the street\nB. other side of the st...
47,"When parking downhill, you should turn your fr...",toward the curb of the sidewalk,away from the curb,any direction will do,NaN,NaN,A,A. toward the curb of the sidewalk\nB. away fr...
16,Which of the following is the maximum speed li...,60 kph,80 kph,100 kph,NaN,NaN,C,A. 60 kph\nB. 80 kph\nC. 100 kph



Holdout Validation Data:


,Question,A,B,C,D,E,Answer,Choices
40,What light shall be used when vehicles are par...,Headlight,Parking lights or lower-beam headlights,Signal lights,NaN,NaN,B,A. Headlight\nB. Parking lights or lower-beam ...
22,To have one's driver's license suspended means...,have it revalidated by the LTO,have it taken away premanently by the LTO,have it taken temporarily by the LTO,NaN,NaN,C,A. have it revalidated by the LTO\nB. have it ...
55,"On a two-lane road, overtaking is only allowed...",left lane,both right and left lane,right lane,NaN,NaN,A,A. left lane\nB. both right and left lane\nC. ...
88,The driver is using a motor vehicle in committ...,revokes and will pay a fine,confiscated and will pay fine,suspended and will pay fine,NaN,NaN,A,A. revokes and will pay a fine\nB. confiscated...
0,What should you do in case your vehicle breaks...,Open your trunk and hood,Stand on the expressway and flag down passing ...,Call for help using a mobile phone or an expre...,Park as far to the right as possible,Put your hazard warning light on,"A, C, D, E",A. Open your trunk and hood\nB. Stand on the e...


In [34]:
df = test_df.copy()
df["AI"] = np.nan
ai_answer = []

for i in tqdm(range(len(df))):
    answ = gen_query(df["Question"].iloc[i], df["Choices"].iloc[i], top_k=20, client=client, mode='hybrid', model="llama3.3")
    ai_answer.append(answ[0])  # Get the first answer

df["AI"] = ai_answer
print("Updated Testing Data with AI Answers:")
display(df.head())

  0%|          | 0/60 [00:00<?, ?it/s]

{
    "main_query": "If the driver is turning left, he must: (U-turn / Left Rule)",
    "sub_queries": [
        {
            "sub_query": "Explain Basic Turning Rules"
        },
        {
            "sub_query": "Describe Left Turn Signals"
        },
        {
            "sub_query": "Outline Conditions for a U-Turn"
        },
        {
            "sub_query": "Discuss Yielding During Left Turns"
        },
        {
            "sub_query": "Explain Intersection Left Turn Procedures"
        },
        {
            "sub_query": "List Common Mistakes in Left Turns"
        }
    ]
}
using Hybrid


  2%|▏         | 1/60 [00:59<58:39, 59.66s/it]

{
    "main_query": "When a vehicle starts to skid, what should the driver do?",
    "sub_queries": [
        {
            "sub_query": "What is the first action a driver should take when the vehicle starts to skid?"
        },
        {
            "sub_query": "How should the driver steer the vehicle during a skid?"
        },
        {
            "sub_query": "What role does braking play when a vehicle is skidding, and how should it be done?"
        },
        {
            "sub_query": "How does vehicle speed affect the likelihood and control of skidding?"
        },
        {
            "sub_query": "What are the differences in handling between oversteer and understeer skids, and how should they be corrected?"
        },
        {
            "sub_query": "What vehicle features or systems can help prevent or mitigate skidding, such as stability control or anti-lock braking?"
        }
    ]
}
using Hybrid


  3%|▎         | 2/60 [01:58<56:56, 58.91s/it]

...

 98%|█████████▊| 59/60 [52:44<00:48, 48.51s/it]

{
    "main_query": "You may never park:",
    "sub_queries": [
        {
            "sub_query": "Understanding Parking Signs"
        },
        {
            "sub_query": "Parking Regulations by Zone"
        },
        {
            "sub_query": "Time Restrictions for Parking"
        },
        {
            "sub_query": "Special Events and Parking"
        },
        {
            "sub_query": "Parking for People with Disabilities"
        },
        {
            "sub_query": "Technological Solutions for Parking Management"
        }
    ]
}
using Hybrid


100%|██████████| 60/60 [53:22<00:00, 53.37s/it]

Updated Testing Data with AI Answers:


,Question,A,B,C,D,E,Answer,Choices,AI
69,"If the driver is turning left, he must: (U-tur...",have the right of way,do so slowly with caution,yield to approaching cars,NaN,NaN,C,A. have the right of way\nB. do so slowly with...,[C]
5,"When a vehicle starts to skid, what should the...",Immediately step on the brakes,Hold firmly on to the wheel while slowing down...,Turn the wheels tp the opposite the direction ...,NaN,NaN,B,A. Immediately step on the brakes\nB. Hold fir...,[B]
42,"If you are parking uphill without a curb, turn...",edge of the street,other side of the street,middle of the street,NaN,NaN,A,A. edge of the street\nB. other side of the st...,[A]
47,"When parking downhill, you should turn your fr...",toward the curb of the sidewalk,away from the curb,any direction will do,NaN,NaN,A,A. toward the curb of the sidewalk\nB. away fr...,[A]
16,Which of the following is the maximum speed li...,60 kph,80 kph,100 kph,NaN,NaN,C,A. 60 kph\nB. 80 kph\nC. 100 kph,[C]


In [35]:
import re


def process_answers(answers):
    formatted_answers = []
    
    for a in answers:
        
        matches = re.findall(r'\[?\s*([A-E](?:\s*,\s*[A-E])*)\s*\]?', str(a)) # Extract answers like [A, C, D] or [A] or [B, D]
        answers = []
        for match in matches:
            answers.extend(re.split(r'\s*,\s*', match))  # Split by comma and remove spaces
        unique_sorted_answers = sorted(set(answers), key=lambda x: ['A', 'B', 'C', 'D', 'E'].index(x))
        if not unique_sorted_answers:
            formatted_answers.append(None)
        else:
            formatted_answers.append(unique_sorted_answers)
    return formatted_answers

df_results = df[["Question", "Answer", "AI"]].copy()
df_results['Answer'] = df_results['Answer'].apply(lambda x: x.split(', '))
df_results['AI'] = process_answers(df_results["AI"])
df_results['Answer'] = process_answers(df_results["Answer"])



def calculate_scores(df):
    scores = []
    for index, row in df.iterrows():
        correct_answers = set(row['Answer'] if row['Answer'] is not None else [])
        ai_answers = set(row['AI'] if row['AI'] is not None else [])
        if ai_answers == correct_answers:
            score = 1.0
        else:
            score = 0.0
        scores.append(score)
    
    df['Score'] = scores
    accuracy = scores.count(1.0) / len(scores)
    print(f'Final Score: {scores.count(1.0):.2f}/{len(scores):.2f}')
    print(f'Accuracy: {accuracy:.2f}%')
    return df

# Apply the scoring function
scored_df = calculate_scores(df_results)

# Display the dataframe to verify the results
display(scored_df[['Question', 'Answer', 'AI', 'Score']])

Final Score: 46.00/60.00
Accuracy: 0.77%


,Question,Answer,AI,Score
69,"If the driver is turning left, he must: (U-tur...",[C],[C],1.0
5,"When a vehicle starts to skid, what should the...",[B],[B],1.0
42,"If you are parking uphill without a curb, turn...",[A],[A],1.0
47,"When parking downhill, you should turn your fr...",[A],[A],1.0
16,Which of the following is the maximum speed li...,[C],[C],1.0
45,Never park or stop at the side of the road wit...,[C],[B],0.0
34,While driving with maximum speed and you have ...,[C],[C],1.0
7,What will happen when your rear tire blows out?,[B],[A],0.0
79,Operating a public utility vehicle equipped wi...,[A],[B],0.0
27,Your speed while driving at night should keep on:,[A],[A],1.0


# Visualization

In [36]:
# Gradio interface with dynamic model, mode selection, and top_k slider
iface = gr.Interface(
    fn=lambda query, top_k, model, mode: gen_query(
        query=query,
        top_k=top_k,
        client=client,
        mode=mode,
        model=model
    ),
    inputs=[
        gr.Textbox(label="Enter your query"),
        gr.Slider(1, 20, value=5, step=1, label="Top K Results"),  # Slider for top_k (1 to 20)
        gr.Dropdown(
            choices=["llama3.3", "llama3.2:latest", "llama3.1:8b"],
            value="llama3.3",
            label="Select Model"
        ),
        gr.Dropdown(
            choices=["hybrid", "dense", "sparse"],
            value="hybrid",
            label="Select Retrieval Mode"
        )
    ],
    outputs=[
        gr.Textbox(label="Answer", lines=6),
        gr.Textbox(label="References", lines=10),
    ],
    title="RAG System with LLaMA Models",
    description=(
        "Ask questions and get answers with references from PDF documents. "
        "Adjust Top-K to control the number of retrieved chunks. "
        "Choose different models and retrieval modes for customization."
    )
)

# Launch the Gradio interface
iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [37]:
# Handle closing behavior
def on_close():
    iface.close()
on_close()

Closing server running on port: 7860
